In [ ]:
import geopandas as gpd
import fiona

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
geodata = gpd.read_file('chapter_12_data.kml')
geodata.head()


In [ ]:
geodata.plot()

In [ ]:
import contextily as cx
ax = geodata.plot(figsize=(15,15), markersize=64)
cx.add_basemap(ax, crs = geodata.crs)


In [ ]:
#Let's start with finding POI of one person
one_person = geodata[geodata['Name'] =='Person 1']
one_person


In [ ]:
ax = one_person.plot(figsize=(15,15), markersize=64)
cx.add_basemap(ax, crs = one_person.crs)


In [ ]:
import pandas as pd
one_person_points_df = pd.DataFrame(
    [x.strip('(').strip(')').strip('0').strip(' ').split(' ') 
    for x in str(one_person.loc[0, 'geometry'])[13:].split(',')],
    columns=['long','lat']
)
one_person_points_df = one_person_points_df.astype(float)
one_person_points_df.head()


In [ ]:
from sklearn.cluster import OPTICS
import numpy as np

clustering = OPTICS(metric='haversine')

one_person_points_df.loc[:,'cluster'] = clustering.fit_predict(np.radians(one_person_points_df[['lat', 'long']]))


In [ ]:
one_person_points_df['cluster'].value_counts()

In [ ]:
# remove all the observations with cluster -1 (outliers)
one_person_points_df = one_person_points_df[one_person_points_df['cluster'] != -1]


In [ ]:
medians_of_POI = one_person_points_df.groupby(['cluster'])[['lat', 'long']].median().reset_index(drop=False)
medians_of_POI


In [ ]:
from shapely.geometry.point import Point
medians_of_POI_gdf = gpd.GeoDataFrame(medians_of_POI, geometry=[Point(x) for x in zip( list(medians_of_POI['long']), list(medians_of_POI['lat']))])
medians_of_POI_gdf.plot()


In [ ]:
ax = one_person.plot(figsize=(15,15))
medians_of_POI_gdf.plot(ax=ax,markersize=128)
cx.add_basemap(ax, crs = one_person.crs)


In [ ]:
# try different settings
one_person_points_df = pd.DataFrame(
    [x.strip('(').strip(')').strip('0').strip(' ').split(' ') 
    for x in str(one_person.loc[0, 'geometry'])[13:].split(',')],
    columns=['long','lat']
)
one_person_points_df = one_person_points_df.astype(float)

clustering = OPTICS(min_samples = 10, max_eps=2.,min_cluster_size=8, xi = 0.05, metric='haversine')

one_person_points_df.loc[:,'cluster'] = clustering.fit_predict(np.radians(one_person_points_df[['lat', 'long']]))
one_person_points_df = one_person_points_df[one_person_points_df['cluster'] != -1]
medians_of_POI = one_person_points_df.groupby(['cluster'])[['lat', 'long']].median().reset_index(drop=False)

print(medians_of_POI)

medians_of_POI_gdf = gpd.GeoDataFrame(medians_of_POI, 
geometry=[Point(x) for x in zip( list(medians_of_POI['long']), list(medians_of_POI['lat']))])

ax = one_person.plot(figsize=(15,15))
medians_of_POI_gdf.plot(ax=ax,markersize=128)
cx.add_basemap(ax, crs = one_person.crs)



In [ ]:
import matplotlib.pyplot as plt

for i,row in geodata.iterrows():
    print(row)
    one_person_points_df = pd.DataFrame(
    [x.strip('(').strip(')').strip('0').strip(' ').split(' ') 
    for x in str(row['geometry'])[13:].split(',')],
    columns=['long','lat']
    )
    one_person_points_df = one_person_points_df.astype(float)

    clustering = OPTICS(min_samples = 10, max_eps=2.,min_cluster_size=8, xi = 0.05, metric='haversine')

    one_person_points_df.loc[:,'cluster'] = clustering.fit_predict(np.radians(one_person_points_df[['lat', 'long']]))

    one_person_points_df =       one_person_points_df[one_person_points_df['cluster'] != -1]

    medians_of_POI = one_person_points_df.groupby(['cluster'])[['lat', 'long']].median().reset_index(drop=False)

    print(medians_of_POI)

    medians_of_POI_gdf = gpd.GeoDataFrame(medians_of_POI, geometry=[Point(x) for x in zip( list(medians_of_POI['long']), list(medians_of_POI['lat']))])

    ax = gpd.GeoDataFrame([row], geometry=[row['geometry']]).plot(figsize=(15,15))

    medians_of_POI_gdf.plot(ax=ax,markersize=128)
    plt.show()
